
<br><font size=6 color=#000000> <b>Batfish project : analysis of the mini-internet configuration</b> </font> <br>

Authors : *Paul-Edouard Bertrand Van Ouytsel* and *Guillaume Steveny*  

Professor : *Olivier Bonaventure*

Teaching assistant : *Thomas Wirtgen*

Academic year : *2022-2023*

______


## Introduction

As part of our UCLouvain classes in computer networking: configuration and management (LINFO2142), we were asked to analyze a new tool to describe a network configuration. This tool is Batfish, an informative tool developed to get more insights of a network and validate its deployment. Easily usable by the mean of its python module able to communicate with a docker container running the backend system of batfish, this tool conveys a lot of useful information about the BGP, OSPF and structures used inside our routers with CISCO or CUMULUS configuration formats.

In order to explore all the possibilities offered by this application and how it could be concretely deployed for real networks. We will work on a mini-internet configuration made earlier during this academic year by the other students attending the course including ourselves. We had to configure a ten of routers inside an AS by specifying them their IP addresses for each interface, their OSPF configuration and their BGP connections (iBGP and eBGP) to allow inter-ASes communication. This had to be done for IPv4 addresses as well as it could have been done for IPv6.

This notebook is targeted to explore the quantity of information provided by the batfish python library. The use of pandas DataFrame embedded in the pybatfish project will be more viewable by the mean of the clear and personalized representation of this jupyter project. We will also try to figure out if such a tool like batfish could be suitable for a kind of automated grading of the configurations made by the students.

The first part will be consecrated to the exploration of the initiation of the sources files, the objects and the parsing of our files. After that, we will discuss the intrinsic structures of the different parts of the networking system and the inner node properties. Then we will focus ourselves to the intra AS networking and the OSPF communication between the router deployed inside an AS. Finally, the BGP point of view of our entire network will be considered. The last part will be a conclusion and a brief overview of the uses of batfish that couldn't have been achieved in our case but worth mentioning.

---



<br><font size=5 color=#000000> <b>PART 1 - INITIATING THE PROJECT</b> </font> <br>

## Deployment of the necessary modules

Here you will find the different imports needed to this project


<div class="alert alert-warning">
<b>[WARNING] Running this notebook</b>  <br>
As we will see later one, the next blocks of code won't be able to run if you didn't deploy a docker container containing the batfish image of the system. This process will be used to generate the answers to all the questions we could ask to our configuration (more on this later). The setup of this infrastructure is quite simple. First, you have to download, from docker hub, the image of this container. To do so, you can use the command [<samp>docker pull allinone/batfish</samp>]. After the previous command succeeds, you will be able to use [<samp>docker run --name batfish -v batfish-data:/data -p 8888:8888 -p 9997:9997 -p 9996:9996 batfish/allinone</samp>]. This notebook supposes that you deployed this container before running further cells of code. In order to make the next cells run, you will also have to use [<samp>python3 -m pip install --upgrade pybatfish</samp>] in order to install the necessary packages for the python interpreter. This jupyter project also needs to have access to the pandas module. So, to avoid any kind of error, please be sure the latest version is correcly installed on your computer.
</div>

In [1]:
#------------------------------------------
#
# Importing of the modules
#
#------------------------------------------

import logging
from typing import List, Optional  # for type hints inside our code

# All the utils for dealing with the answers given by pybatfish
import pandas as pd

# Import numpy for some data processing
import numpy as np

# Function to pretty print the DataFrames
from pandas.io.formats.style import Styler

# Class to create a computing session of the Docker container of Batfish
from pybatfish.client.session import Session  # noqa: F401

# Some useful utils contained inside pybatfish
from pybatfish.datamodel import Edge, Interface
from pybatfish.datamodel.answer import TableAnswer
from pybatfish.datamodel.flow import HeaderConstraints, PathConstraints
from pybatfish.datamodel.route import BgpRoute, BgpRouteConstraints
from pybatfish.util import get_html


### Instantiation of pandas and logging module


In [2]:
# Configure all pybatfish loggers to use WARN level
logging.getLogger("pybatfish").setLevel(logging.WARN)

# Avoid to have limits of display for the DataFrames
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

# Prevent rendering text between '$' as MathJax expressions
pd.set_option("display.html.use_mathjax", False)


### Styler for presenting DataFrames


In [3]:
_STYLE_UUID = "pybfstyle"


class MyStyler(Styler):
    """A custom styler for displaying DataFrames in HTML"""

    def __repr__(self):
        return repr(self.data)


def show(df):
    """
    Displays a dataframe as HTML table.

    Replaces newlines and double-spaces in the input with HTML markup, and
    left-aligns the text.
    """
    if isinstance(df, TableAnswer):
        df = df.frame()

    # workaround for Pandas bug in Python 2.7 for empty frames
    if not isinstance(df, pd.DataFrame) or df.size == 0:
        display(df)
        return
    display(
        MyStyler(df)
        .set_uuid(_STYLE_UUID)
        .format(get_html)
        .set_properties(**{"text-align": "left", "vertical-align": "top"})
    )

---

## Creating the pybatfish session object

In order to further analyse our network, we first need to create a Session object which will contain a snapshot of the configuration files included in our network. This Session is a way for the pybatfish module to communicate with our batfish server that have to be deployed before running the code. This server, deployed as a docker container, will be responsible for the computation asked our system.

But to be able to actually work on our current configuration, we have to specify the path of the directory containing a `configs` directory which itself contains the configuration files for each router. The snippet of code below instantiate our actual network.


In [4]:
# Batfish session object
bf = Session(host="localhost")

# Variables of our studied network
NETWORK_NAME = "network"
SNAPSHOT_NAME = "snapshot"
SNAPSHOT_PATH = "global_network/"

# Setting the network name
bf.set_network(NETWORK_NAME)

# Creation of the snapshot of our network to analyse it
bf.init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'snapshot'


### Overview of the configurations

Then this object can be used by the mean of batfish question. This part of the Session objects allows to get more information concerning features of our network such as the router names, the possible routes or event the structures having some trouble to be parsed by the pybatfish module.

Here you can find the complete parse status of the files involved in the snapshot discussed in the previous code cell.


In [5]:
bf.q.fileParseStatus().answer().frame()

,File_Name,Status,File_Format,Nodes
0,configs/BASE_24.cfg,PASSED,CUMULUS_CONCATENATED,['base_router_24']
1,configs/BASE_25.cfg,PASSED,CUMULUS_CONCATENATED,['base_router_25']
2,configs/BASE_26.cfg,PASSED,CUMULUS_CONCATENATED,['base_router_26']
3,configs/BASE_27.cfg,PASSED,CUMULUS_CONCATENATED,['base_router_27']
4,configs/BASE_28.cfg,PASSED,CUMULUS_CONCATENATED,['base_router_28']
5,configs/BASE_29.cfg,PASSED,CUMULUS_CONCATENATED,['base_router_29']
6,configs/BASE_30.cfg,PASSED,CUMULUS_CONCATENATED,['base_router_30']
7,configs/GENE_24.cfg,PASSED,CUMULUS_CONCATENATED,['gene_router_24']
8,configs/GENE_25.cfg,PASSED,CUMULUS_CONCATENATED,['gene_router_25']
9,configs/GENE_26.cfg,PASSED,CUMULUS_CONCATENATED,['gene_router_26']



What we see here is that, happily, every file was correctly understood by the system and will be correctly analysable.

> This behaviour wasn't the one we get when we started this project. Indeed, our configuration files weren't either of CISCO format or CUMULUS format leading to a lot of problems to parse the files. The syntax was unrecognized and nothing was answered to the different questions asked to pybatfish. So we took time to do a "manual" conversion of each file implied in our network. By the mean of a separate python script, the configurations were translated, one by one, by verifying inside the docs of CUMULUS which instructions are recognized and what are the syntax rules accepted by batfish. After discussion with our professor Olivier Bonaventure, we decided to get rid of the IPv6 part of the mini-internet. This addressing scheme wasn't mandatory and a lot of students did not take the time to do it. So, during the rest of this notebook, only IPv4 will be used to analyze the results.

<div class="alert alert-danger">
<b>[CAUTION] Configuration type</b>  <br>
Take care of the format you use to feed batfish. Be sure to use a correct configuration file respecting the CUMULUS, CISCO or ARISTA formats. Only using files corresponding to your FRRouting configuration won't let you completly use the tool. Even with some syntax errors the complete analysis process could not have been correctly realised and the results could be completly unreadable. During our translation of the different files we encountered a lot of problems. Ths process isn't as easy as we may think because some correct instructions according to the FRRouting documentation, weren't recognized by batfish. For some of them, we didn't find a way to go around them and use an equivalent instruction. So, if batfish was used to grade the config files, some students could have been badly graded even if their system was doing the job. This is one first big limit of this system. So, if you decide to use our translation python script on other configuration files, we don't guarantee a complete and correct translation. All the files used in this notebook were translated correctly, but the huge amount of configuration possibilities doesn't let us cover all the cases.
</div>

For readability, we limited ourselves to this subset of our entire internet network (which was composed of nearly 70 ASes of 8 routers). This will allow us to get results more suitable for exploring the power of batfish. Furthermore, the more we add configurations, the more answering to questions take time. This could be seen as a little inconvenient of batfish: the time needed to answer to some questions, especially the one we could use to do virtual traceroutes. If we want to grade automatically the student, we such have to be sure to have the resources and time to run the questions for the entire network.

The next code snippet validates that there is no parsing error while handling our configuration files.

In [6]:
bf.q.parseWarning().answer().frame()

,Filename,Line,Text,Parser_Context,Comment



Everything was understood by the system, our configuration are well formatted

<div class="alert alert-info">
<b>[Remark] Using question for pybatfish</b><br>
You should have noticed now the particular form of questions asked to our Session object. We thought an brief overview of each component could be worth it. First the q field indicates we cant to ask a question to pybatfish. Then comes the desired question (fileParseStatus, parseWarning, ...), followed by the indication that we want for the module to answer to this question (returning a JSON object). If we don't specify this, the answer is not computed, only kept in a lazy way. Finally the frame function allows to transform this answer JSON object to a readable pandas DataFrame. 
</div>


### Error injections

A more interesting way to discover batfish and its power to validate our configurations could be, instead of saying that our reformatted files are recognized, to inject some errors of syntax, configuration or whatever to correctly see if this tool is able to catch them. This could lead to great improvements for the work of thousands of network developer. With a simple run of batfish, we would be directly advertised of a problem instead of hour of debugging. The fat fingers problem is something really tedious to quickly discover when lines of code have to be carefully studied.

The first error type we will inject is a simple syntax error. We will create a new snapshot containing the erroneous configuration file. The unlucky modified one was the BASE_router configuration of the AS number 27. To be sure that batfish correctly discover all mistakes we injected, we will identify each of them before running the code section.

- line 7: `auto BASE-L2.10` became `autos BASE-L2.10`
- line 28: `iface port_GENE inet static` became `iwace port_GENE inet static`
- line 13: `address 27.192.0.1/12` became `address 27.192.0.1/67`
- line 52: `neighbor 25.43.6.2 remote-as 25` became `neighbor 25.43.6.2 remote-as 2 5`
- line 65: `neighbor 27.144.7.0 update-source lo` became `neighbour 27.144.7.0 update-source lo`
- line 89: `bgp community-list standard 1 permit 27:100` became `bgp community-list standard 1 permit 27;100`

We also decided to change the name of the router inside its configuration to see if it leads to collisions and thus problems with the comprehension of our system by batfish. So inside BASE_router, at the line 1, `BASE_router_27` became `GENE_router_27` which it's the same name as the "GENE" router configuration.

This simple set of errors was chosen because we thought it would describe some comme syntax and unexpected mistakes a programmer could make if he was deploying its network.

In [7]:
# Batfish session object
bf_syntaxError = Session(host="localhost")

# Variables of our studied network
NETWORK_NAME = "syntax_network"
SNAPSHOT_NAME = "syntax_snapshot"
SNAPSHOT_PATH = "syntax_error/"

# Setting the network name
bf_syntaxError.set_network(NETWORK_NAME)

# Creation of the snapshot of our network to analyse it
bf_syntaxError.init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

Your snapshot was successfully initialized but Batfish failed to fully recognized some lines in one or more input files. Some unrecognized configuration lines are not uncommon for new networks, and it is often fine to proceed with further analysis. You can help the Batfish developers improve support for your network by running:

    bf.upload_diagnostics(dry_run=False, contact_info='<optional email address>')

to share private, anonymized information. For more information, see the documentation with:

    help(bf.upload_diagnostics)


'syntax_snapshot'

The first interesting to already considered is this warning message kindly given by pybatfish when he tried to initiate our snapshot. Here because we use a problematic file, it leads to a result we expected. By rather than sending our stupid falsified network to the developers, we will analyse the errors discovered by their tool.

In [8]:
bf_syntaxError.q.fileParseStatus().answer().frame()

,File_Name,Status,File_Format,Nodes
0,configs/BASE_27.cfg,PARTIALLY_UNRECOGNIZED,CUMULUS_CONCATENATED,"['gene_router_27__configs__base_27.cfg', 'gene_router_27__configs__base_27.cfg']"
1,configs/GENE_27.cfg,PASSED,CUMULUS_CONCATENATED,"['gene_router_27', 'gene_router_27__configs__gene_27.cfg']"
2,configs/LUGA_27.cfg,PASSED,CUMULUS_CONCATENATED,['luga_router_27']
3,configs/LYON_27.cfg,PASSED,CUMULUS_CONCATENATED,['lyon_router_27']
4,configs/MILA_27.cfg,PASSED,CUMULUS_CONCATENATED,['mila_router_27']
5,configs/MUNI_27.cfg,PASSED,CUMULUS_CONCATENATED,['muni_router_27']
6,configs/VIEN_27.cfg,PASSED,CUMULUS_CONCATENATED,['vien_router_27']
7,configs/ZURI_27.cfg,PASSED,CUMULUS_CONCATENATED,['zuri_router_27']


The first step done above is to check the parsing status of our files. We directly see the location of the problematic one: BASE_27. There are three kind of status inside the parsing step: unrecognized, partially_recognized and passed. But these are only leading to warnings. The rest of our code will continue to work but will just ignore the problematic configuration for getting the desired information that could not have been parsed. Thus, we can print the warnings to identify where we should correct our syntax.

In [9]:
bf_syntaxError.q.parseWarning().answer().frame()

,Filename,Line,Text,Parser_Context,Comment
0,configs/BASE_27.cfg,7,autos BASE-L2.10,[s_auto statement cumulus_interfaces_configuration],This syntax is unrecognized
1,configs/BASE_27.cfg,13,address 27.192.0.1/67,[i_address i_property si_inet s_iface statement cumulus_interfaces_configuration],This syntax is unrecognized
2,configs/BASE_27.cfg,28,iwace port_GENE inet static,[cumulus_interfaces_configuration],This syntax is unrecognized
3,configs/BASE_27.cfg,29,address 27.128.32.1/31,[cumulus_interfaces_configuration],This syntax is unrecognized
4,configs/BASE_27.cfg,52,neighbor 25.43.6.2 remote-as 2 5,[rbnp_remote_as rbn_property rbn_ip rb_neighbor bgp_inner s_router_bgp statement frr_configuration],This syntax is unrecognized
5,configs/BASE_27.cfg,65,neighbour 27.144.7.0 update-source lo,[s_router_bgp statement frr_configuration],This syntax is unrecognized
6,configs/BASE_27.cfg,89,bgp community-list standard 1 permit 27;100,[literal_standard_community standard_community bcl_standard b_community_list s_bgp statement frr_configuration],This syntax is unrecognized


Surprisingly, or not, batfish found more errors than the number of injected ones. Could it be a flaw in its functionality ? The answer is obviously no. Let's look at the third line of the previous frame. The error is said to be located inside the 29th line, just after the error injected at the 28th line. This is so, also an injected error. By transforming `iface` to `iwace` we led to a second error where the address giving line was orphan of its related interface. So having an address out of nowhere like this in our code, is an error correctly discovered.

<div class="alert alert-info">
<b>[Remark] Fat finger problems</b><br>
The error shown in the previous example have more of an illustrating purpose than a desribing one. Nowadays, most configuration are written thanks to command line and not from scratch inside txt files. When problems like above are typed inside the terminal, the configuration tool is really likely to throw an error itself to say that a syntax is unrecognized. Nonetheless, the capacity of batfish to also detect it was worth mentioning. After all, a corrumption of the information stored for the configurations could always happen and thus leads to these kinds of problems.
</div>

What is also interesting to notice is the ability of batfish to resolve conflicting names. Indeed, no warning was produced to say that two routers share the same name. We could see inside the parse status of our files that our tool directly assigned a different name to each router. But we have to be careful with this conflict resolving technique to be sure that this doesn't create more problems during the next analyses.

<div class="alert alert-warning">
<b>[Warning] Wiping out our first snapshot</b><br>
You maybe noticed that our instanciation of our snapshot was a little different that our first one. Indeed, the names are specific for this case. This allows us to keep in memory of the batfish container the two current configurations. We don't have to initialize only one system. This could be useful when we want to compare two possible networks for example.
</div>

<br><font size=5 color=#000000> <b>PART 2 - INNER NODE PROPERTIES</b> </font> <br>

Now we have our system initialized and our questions can be answered, we could want to have some information about the nodes inside our network. Moreover, we could also check the properties of the interfaces and the structures used. This is the aim of this section.

Let's start with a first basic question `nodeProperties`. Its task is simple: showing to us every piece of information about the properties of the nodes contained in our network.

In [10]:
prop_all = bf.q.nodeProperties().answer().frame()
display(prop_all.columns)

Index(['Node', 'AS_Path_Access_Lists', 'Authentication_Key_Chains',
       'Community_Match_Exprs', 'Community_Set_Exprs',
       'Community_Set_Match_Exprs', 'Community_Sets', 'Configuration_Format',
       'DNS_Servers', 'DNS_Source_Interface', 'Default_Cross_Zone_Action',
       'Default_Inbound_Action', 'Domain_Name', 'Hostname', 'IKE_Phase1_Keys',
       'IKE_Phase1_Policies', 'IKE_Phase1_Proposals', 'IP6_Access_Lists',
       'IP_Access_Lists', 'IPsec_Peer_Configs', 'IPsec_Phase2_Policies',
       'IPsec_Phase2_Proposals', 'Interfaces', 'Logging_Servers',
       'Logging_Source_Interface', 'NTP_Servers', 'NTP_Source_Interface',
       'PBR_Policies', 'Route6_Filter_Lists', 'Route_Filter_Lists',
       'Routing_Policies', 'SNMP_Source_Interface', 'SNMP_Trap_Servers',
       'TACACS_Servers', 'TACACS_Source_Interface', 'VRFs', 'Zones'],
      dtype='object')

We choose to display only the list of properties we could get. Since most of them weren't suited to our mini-internet project, the results we could get were not worth continuing the discussion. The only useful information we could extract from this analysis could be the list of routing policies (property `Routing_Policies`) but this will be covered during the section consecrated to OSPF and BGP. Instead, we will focus about the feedback we could have by analyzing the interfaces.

The first thing to do in order to analyse these properties is to list all of them to only select the useful one and thus avoiding too big dataframes. More information about each item of the next list could be found on the batfish documentation: https://batfish.readthedocs.io/en/latest/notebooks/configProperties.html#Interface-Properties

In [11]:
iface_all = bf.q.interfaceProperties().answer().frame()
display(iface_all.columns)

Index(['Interface', 'Access_VLAN', 'Active', 'Admin_Up', 'All_Prefixes',
       'Allowed_VLANs', 'Auto_State_VLAN', 'Bandwidth', 'Blacklisted',
       'Channel_Group', 'Channel_Group_Members', 'DHCP_Relay_Addresses',
       'Declared_Names', 'Description', 'Encapsulation_VLAN', 'HSRP_Groups',
       'HSRP_Version', 'Inactive_Reason', 'Incoming_Filter_Name', 'MLAG_ID',
       'MTU', 'Native_VLAN', 'Outgoing_Filter_Name', 'PBR_Policy_Name',
       'Primary_Address', 'Primary_Network', 'Proxy_ARP', 'Rip_Enabled',
       'Rip_Passive', 'Spanning_Tree_Portfast', 'Speed', 'Switchport',
       'Switchport_Mode', 'Switchport_Trunk_Encapsulation', 'VRF',
       'VRRP_Groups', 'Zone_Name'],
      dtype='object')

So, we decided to limit the properties to a smaller set. We selected `Active` to see if an interface was configured and able to process packets. This first information could be useful to see if a student forget to deploy an interface or simply broke it by any kind of manipulation. Secondly, we also chose `Primary_Address` which is important to see the addressing scheme used by the student. Finally, the `Bandwidth` is not really part of the student configurations but will be useful for a little note later on.

In [12]:
iface_prop = bf.q.interfaceProperties(properties="Bandwidth,Active,Primary_Address").answer().frame()
display(iface_prop)

,Interface,Active,Bandwidth,Primary_Address
0,base_router_24[BASE-L2],True,1e+10,None
1,base_router_24[BASE-L2.10],True,1e+10,24.1.0.2/20
2,base_router_24[BASE-L2.20],True,1e+10,24.2.0.2/20
3,base_router_24[ext_22_ZURI],True,1e+10,179.0.11.2/24
4,base_router_24[host],True,1e+10,24.0.20.2/31
...,...,...,...,...
406,zuri_router_30[port_MUNI],True,1e+10,30.0.4.1/24
407,zuri_router_30[port_VIEN],True,1e+10,30.0.5.1/24
408,zuri_router_30[ZURI-L2],True,1e+10,None
409,zuri_router_30[ZURI-L2.10],True,1e+10,30.200.10.1/24


We thus see the interest of the `properties` argument inside the previous question. We can limit to a subset quite easily. By specifying this string where each property is "comma" separated, we could limit ourselves to this readable dataframe. This has also an advantage to limit the computation time for this question because of the "lazy" processing of batfish that only compute what it needs.

A first feedback that could be useful for some students, is to list the inactive interfaces.

In [13]:
iface_prop[iface_prop["Active"] == False]

,Interface,Active,Bandwidth,Primary_Address


As we can see, nothing is displayed inside our dataframe. Ths means that every student inside our little subset of the network mess with the interfaces in order to disable them. However, a more interesting criterion to analyse could be to list all the interfaces lacking of an address. This could a first indicator of what the student have done and what remains to be achieved for its network to communicate effectively with its peers. The next code snippet allows to filter all the interfaces with no address.

In [14]:
iface_prop[iface_prop["Primary_Address"].astype(str) == "None"]

,Interface,Active,Bandwidth,Primary_Address
0,base_router_24[BASE-L2],True,1e+10,None
43,base_router_28[port_GENE],True,1e+10,None
44,base_router_28[port_LYON],True,1e+10,None
45,base_router_28[port_MUNI],True,1e+10,None
46,base_router_28[port_ZURI],True,1e+10,None
48,base_router_29[BASE-L2],True,1e+10,None
58,base_router_30[BASE-L2],True,1e+10,None
68,gene_router_24[GENE-L2],True,1e+10,None
77,gene_router_25[GENE-L2],True,1e+10,None
109,gene_router_28[port_BASE],True,1e+10,None


Here the results need a little discussion since the conclusions are not straightforward in this case. We are faced to two principal problems. The interfaces containing L2 in their names and some interfaces of the AS 28 (identified by the routers' names containing 28).

The first case (L2 interfaces) is quite more understandable when we know the kind of interface they were bound to. Indeed, these structures were used to link our routers named base, zuri and gene to a subnetwork of switches to simulate a datacenter. There were two types of tag involved in this center: 10 and 20 allowing the students to define a virtual network where some hosts could be directly joined without having to go through a router. In order to set the tags, each of the specified routers had 2 interfaces which contains L2.10 and L2.20 to specify the tag that had to be attached for packets flowing through them. But since no default route has to be implemented, some students simply chose to leave empty the interface corresponding to this route. This explains the L2 case. 

But for understanding what the student responsible for the AS28 does, we need to delve more into the details of its configuration. Listing all the interfaces properties of one of his/her router could help us to see where the problem comes from.

In [15]:
bf.q.interfaceProperties(nodes='luga_router_28',properties="Bandwidth,Active,Primary_Address").answer().frame()

,Interface,Active,Bandwidth,Primary_Address
0,luga_router_28[ext_27_LUGA],True,1e+10,179.0.69.2/24
1,luga_router_28[GENE],True,1e+10,28.0.9.2/24
2,luga_router_28[host],True,1e+10,28.104.0.0/24
3,luga_router_28[lo],True,8e+09,28.154.0.1/24
4,luga_router_28[MILA],True,1e+10,28.0.11.1/24
5,luga_router_28[port_GENE],True,1e+10,None
6,luga_router_28[port_MILA],True,1e+10,None
7,luga_router_28[port_VIEN],True,1e+10,None
8,luga_router_28[port_ZURI],True,1e+10,None
9,luga_router_28[VIEN],True,1e+10,28.0.12.1/24


We can thus see that specifying the argument `nodes` with a string representing the focus node limit our analysis to this particular router. The results then give a clear conclusion to this problem: the student defined GENE as its interface which should have been named port_GENE. Indeed, when we check the configuration files of this student, we see two times the same address both for different interface names. This leads to a problem with batfish which wasn't able to understand this fact. Instead, our software decided to consider only the first one to be the correct one leaving the second empty. While not creating big problem here (the interfaces used will be the ones without port inside their name), this could have been helpful for the student while he/she was configuring his/her network. We understand that he/she first defined the incorrect interfaces after switching to the correct ones without deleting the faulting elements. If a feedback given by batfish could have been make earlier, he/she could have been alerted of such a problem quicker.

Before continuing our analyses, we wanted to come back on the `Bandwidth`property we decided to keep while not showing a great interest. This was only to prepare this little discussion. Indeed, our overview of the batfish possibilities is limited to our network but also by the constraint this project involved. If we had more time, we could explore more in details some additional information we could inject inside our snapshot. The bandwidth parameter is one of them. By using what's called `runtime interface information` we could add everything scenarios on the links composing the infrastructure. The main purpose of these files (documented here: https://batfish.readthedocs.io/en/latest/formats.html#runtime-interface-information) is to add this bandwidth information, as well as deciding which link between interfaces has failed or simply what is the delay for transmitting packets over a connection. This could be use for further testing of our network like verifying that the students correctly configured their AS to be sure that a failure doesn't lead to a complete fault inside other ASes or simply if the student put the correct costs on the interfaces if this was part of the project. Since this wasn't the case here, we decided only to introduce the concept by this little overview of such additional files.

<div class="alert alert-info">
<b>[Remark] More additional files</b><br>
But this kind of files isn't the only one offered by batfish. The documentation lists plenty of additional information that could be provided to our snapshot for a better description of the current system. We decided to highlight two of them: the hosts configurations and ISP settings. These two are formatted as JSON files. The first allows us to specify every piece of information each host on each router should contain. While the later serves to set up an ISP by describing how the peerings should take place for example. To see all the possibilities of these files, you can go follow this link: <a href=https://batfish.readthedocs.io/en/latest/formats.html#modeling-hosts>modelling hosts</a> and this one: <a href=https://batfish.readthedocs.io/en/latest/formats.html#modeling-isps> modelling ISP</a>.
</div>

### Structures problems

But now we have the basic information about the properties of our nodes and interfaces, we could want to get more insights about the structures used by the routers. Batfish can also do this. The question `definedReferences` will list for each node all the structures contained in them which could be the route-maps, the interfaces, the community lists and so on. However, this part of our work will be more focused on the problems of structures. Effectively, pybatfish offers question to identify the undefined references (`undefinedReferences`) as well as the unused structures (`unusedStructures`). Each of these cases can lead to some problem we will discuss now.

Let's start with the more horrific one: the undefined structures.

In [16]:
# List references to undefined structures
bf.q.undefinedReferences().answer().frame()

,File_Name,Struct_Type,Ref_Name,Context,Lines
0,configs/LUGA_27.cfg,bgp as-path access-list,IN7,route-map match as-path,configs/LUGA_27.cfg:[83]
1,configs/LUGA_27.cfg,bgp as-path access-list,OUT7,route-map match as-path,configs/LUGA_27.cfg:[87]


This is particularly concerning. The student responsible for the AS27 messed up with the LUGA configuration by using two access lists which aren't defined. This is a serious error. But surprisingly, during our project, this student had a completely functional AS. How could it be possible to use something that our router does not know ? In fact, they don't so the reason have to be somewhere else. We will let this problem apart for now since an explanation is expected to be found in the next few cells.

However, except for this interesting problem located inside the AS27, we can see that all the other students managed well their routers. Let's know focus on the unused structures. Having unused structures doesn't lead to a problem like the one mentioned above but could be as much critical. When a router lacks of memory space, most of the time, it fails and simply reboots. This could create infinite loop where a router will be again completely full after its restart and then failing again. So we have to limit the cases where such problem could arise. Limiting the structures we use to the minimal required is part of the solution. Moreover, this leads to better maintaining of the nodes.

In [17]:
# List references to unused structures
bf.q.unusedStructures().answer().frame()

,Structure_Type,Structure_Name,Source_Lines
0,route-map,ACCEPT_ALL,configs/BASE_25.cfg:[76]
1,route-map,ACCEPT_ALL,configs/BASE_26.cfg:[86]
2,route-map,ACCEPT_ALL,configs/LUGA_25.cfg:[76]
3,route-map,ACCEPT_ALL,configs/LUGA_26.cfg:[74]
4,route-map,ACCEPT_ALL,configs/LUGA_27.cfg:[80]
5,route-map,LUGA_IN,"configs/LUGA_27.cfg:[82, 83, 84]"
6,route-map,LUGA_OUT,"configs/LUGA_27.cfg:[86, 87]"
7,route-map,ACCEPT_ALL,configs/LYON_25.cfg:[68]
8,route-map,MAP_OUT,"configs/LYON_26.cfg:[76, 77]"
9,bgp community-list standard,1,configs/LYON_27.cfg:[71]


What we see again, surprisingly or not, is the large amount of entries concerning the routers inside the AS27. We will focus on them to understand this case.

In [18]:
# List references to unused structures inside the AS27
bf.q.unusedStructures(nodes='/27/').answer().frame()

,Structure_Type,Structure_Name,Source_Lines
0,route-map,ACCEPT_ALL,configs/LUGA_27.cfg:[80]
1,route-map,LUGA_IN,"configs/LUGA_27.cfg:[82, 83, 84]"
2,route-map,LUGA_OUT,"configs/LUGA_27.cfg:[86, 87]"
3,bgp community-list standard,1,configs/LYON_27.cfg:[71]
4,ip prefix-list,ME,configs/LYON_27.cfg:[69]
5,route-map,OUTF,configs/LYON_27.cfg:[79]
6,route-map,ACCEPT_ALL,configs/MUNI_27.cfg:[68]
7,route-map,ACCEPT_ALL,configs/VIEN_27.cfg:[71]
8,route-map,MEO,"configs/VIEN_27.cfg:[82, 83]"
9,route-map,TESTO,"configs/VIEN_27.cfg:[85, 86]"


In fact, one of us was the responsible for the AS27 and created these problems without being aware of them or knowing it would have to explain them later on. So the interpretation of the previous result is quite easier to get since we don't have to delve inside the configuration the student done. Indeed, the main reason of these entries is a lack of perfect cleaning of our configurations. The student was testing a lot of stuff to make its configuration works and this leads to an interesting number of structures that were like a one-shot for a test. Now, he could collect this feedback on its routers, he just wants to go back in time to avoid having such a mess inside his files.

But the fact we pinned out this AS inside the unused structures part is also to answer to the question about the undefined structures. If we go back on our previous table listing these warnings, we could see these problems were located at the lines 83 and 87 of the "LUGA" router. But these lines are also part of the above dataframe. The entries number 1 and 2 cover these lines. So a solution have been found, the structures the student didn't define are used inside structures he did not exploit which don't lead to an error. So, he does some cleaning, but not carefully enough as we can see.

But after this mea culpea of one of us, we wanted to highlight another wat to specify the `nodes` field inside the questions. Indeed, this could be done by using regex expression. The following code snippet shows an example of usage of them.

In [19]:
# List references to unused structures for the AS24 and AS26
bf.q.unusedStructures(nodes='/24|26/').answer().frame()

,Structure_Type,Structure_Name,Source_Lines
0,route-map,ACCEPT_ALL,configs/BASE_26.cfg:[86]
1,route-map,ACCEPT_ALL,configs/LUGA_26.cfg:[74]
2,route-map,MAP_OUT,"configs/LYON_26.cfg:[76, 77]"
3,route-map,MAP-OUT,"configs/MILA_26.cfg:[72, 73]"
4,route-map,ACCEPT_ALL,configs/MUNI_26.cfg:[66]


As we can see, the other member of our team (AS26) also had some unused structures while the AS24 was completely out of these problems.

<br><font size=5 color=#000000> <b>PART 3 - INTRA AS COMMUNICATION</b> </font> <br>

In this section we'll focus on the first part of the mini-internet project : the internal configuration of an AS. We'll see the different ways in which batfish can be used to analyze the configuration of an autonomous system.

### Layer 3 edges

One of the first things we might want to analyze when looking at the internal configuration of an autonomous system is the layer 3 edges. Indeed, not configuring them correctly can have a terrible effect on the network and might even make it unusable. Batfish provides an easy way to check the layer 3 edges of a network.

In [20]:
layer3 = bf.q.layer3Edges().answer().frame()
show(layer3)

,Interface,IPs,Remote_Interface,Remote_IPs
0,base_router_24[BASE-L2.10],24.1.0.2,zuri_router_24[ZURI-L2.10],24.1.0.1
1,base_router_24[BASE-L2.20],24.2.0.2,zuri_router_24[ZURI-L2.20],24.2.0.1
2,base_router_24[port_GENE],24.0.23.2,gene_router_24[port_BASE],24.0.23.3
3,base_router_24[port_LYON],24.0.26.2,lyon_router_24[port_BASE],24.0.26.3
4,base_router_24[port_MUNI],24.0.25.2,muni_router_24[port_BASE],24.0.25.3
5,base_router_24[port_ZURI],24.0.12.3,zuri_router_24[port_BASE],24.0.12.2
6,base_router_25[port_GENE],25.0.6.2,gene_router_25[port_BASE],25.0.6.3
7,base_router_25[port_LYON],25.0.8.2,lyon_router_25[port_BASE],25.0.8.6
8,base_router_25[port_MUNI],25.0.7.2,muni_router_25[port_BASE],25.0.7.5
9,base_router_25[port_ZURI],25.0.1.2,zuri_router_25[port_BASE],25.0.1.1


By default, this query returns all the edges in the network. As usual, we can use a regex to filter the information we want to get. Here, we'll restrict our search to the 27th autonomous system. The student it was assigned to is a member of our group, this will make it easier for us to see if the result correspond to the actual configuration.

In [21]:
layer3Filtered = bf.q.layer3Edges(nodes="/27/").answer().frame()
show(layer3Filtered)

,Interface,IPs,Remote_Interface,Remote_IPs
0,base_router_27[BASE-L2],27.160.0.1,zuri_router_27[ZURI-L2],27.160.0.0
1,base_router_27[BASE-L2.10],27.176.0.1,zuri_router_27[ZURI-L2.10],27.176.0.0
2,base_router_27[BASE-L2.20],27.192.0.1,zuri_router_27[ZURI-L2.20],27.192.0.0
3,base_router_27[ext_25_LYON],25.43.6.3,lyon_router_25[ext_27_BASE],25.43.6.2
4,base_router_27[port_GENE],27.128.32.1,gene_router_27[port_BASE],27.128.32.0
5,base_router_27[port_LYON],27.128.62.1,lyon_router_27[port_BASE],27.128.62.0
6,base_router_27[port_MUNI],27.128.52.1,muni_router_27[port_BASE],27.128.52.0
7,base_router_27[port_ZURI],27.128.21.0,zuri_router_27[port_BASE],27.128.21.1
8,gene_router_27[port_BASE],27.128.32.0,base_router_27[port_GENE],27.128.32.1
9,gene_router_27[port_LUGA],27.128.43.1,luga_router_27[port_GENE],27.128.43.0


This indeed corresponds to the layer 3 edges of AS 27.

### Ospf configuration

While being able to have an overview of the layer 3 edges is already interesting in itself, there is a lot more information about the internal configuration of our AS we could potentially get from our configuration files. One very important aspect of an AS is its internal routing. For the mini-internet project, we were required to use OSPF. The configuration of this protocol on the routers of a network can be easily analyzed by batfish.

The "ospfSessionCompatibility" function gives us the list of all compatible ospf sessions. A session between two routers is considered to be compatible when both routers belong to the same area, have enabled OSPF, are not OSPF-passive and don't have their interfaces down.

In [22]:
bf.q.ospfSessionCompatibility(nodes="/27/").answer().frame()

,Interface,VRF,IP,Area,Remote_Interface,Remote_VRF,Remote_IP,Remote_Area,Session_Status
0,base_router_27[BASE-L2.10],default,27.176.0.1,0,zuri_router_27[ZURI-L2.10],default,27.176.0.0,0,ESTABLISHED
1,gene_router_27[port_LUGA],default,27.128.43.1,0,luga_router_27[port_GENE],default,27.128.43.0,0,ESTABLISHED
2,zuri_router_27[ZURI-L2],default,27.160.0.0,0,base_router_27[BASE-L2],default,27.160.0.1,0,ESTABLISHED
3,base_router_27[BASE-L2.20],default,27.192.0.1,0,zuri_router_27[ZURI-L2.20],default,27.192.0.0,0,ESTABLISHED
4,vien_router_27[port_ZURI],default,27.128.71.0,0,zuri_router_27[port_VIEN],default,27.128.71.1,0,ESTABLISHED
5,luga_router_27[port_ZURI],default,27.128.41.0,0,zuri_router_27[port_LUGA],default,27.128.41.1,0,ESTABLISHED
6,zuri_router_27[port_BASE],default,27.128.21.1,0,base_router_27[port_ZURI],default,27.128.21.0,0,ESTABLISHED
7,zuri_router_27[dns_27],default,198.0.0.27,0,zuri_router_29[dns_29],default,198.0.0.29,0,ESTABLISHED
8,zuri_router_27[port_LUGA],default,27.128.41.1,0,luga_router_27[port_ZURI],default,27.128.41.0,0,ESTABLISHED
9,base_router_27[port_LYON],default,27.128.62.1,0,lyon_router_27[port_BASE],default,27.128.62.0,0,ESTABLISHED


At first glance, the results seem to be what we would expect of this function. We can see that ospf was correctly enabled on the different routers. However, some of the lines seem strange. Indded, we can see that there are OSPF sessions between routers belonging to different ASes ! The zuri routers of the various ASes we have seem to be interconnected. This is actually just due to the way the project was set up and doesn't have anything to do with the way students configured their ASes or the way batfish performs its computations.

### Performing traceroutes

Now that we know how our autonomous system was configured, it's time to see how it works in practice. A good way to do so is to use a traceroute to see the different routers a packet goes through before reaching its destination. We can then compare the result batfish gives us with what we expect.

In this example, we try to do a traceroute from the LYON router to the VIEN router (still in AS 27)

In [23]:
LyonZuriTrace = bf.q.traceroute(startLocation="lyon_router_27", headers=HeaderConstraints(dstIps="27.144.7.0", srcIps="27.144.6.0")).answer().frame()
show(LyonZuriTrace)

Flow  \
0  start=lyon_router_27 [27.144.6.0:49152->27.144.7.0:33434 UDP]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

After verifying manually, we can see that the 6 routes we got from batfish are all valid and go trough 4 routers.

<div class="alert alert-info">
<b>[REMARK] Bi-directional traceroute</b><br>
    It's important to note that the default traceroute query <b>bf.q.traceroute()</b> doesn't entirely follow the behavior of the traceroute command. Indeed, for the tracereoute command to work correctly, the packets exchanged by the two hosts need to be able to go both ways. This is not the case of this function, which only tests the connection in one way. If you want to make sure packets are able to go both ways, you should use the <b>bf.q.bidirectionalTraceroute()</b> query.
</div>

### Routing tables

Another interesting element of our configurations we could have a look at are the routing tables of the various routers of an AS. They define the way a router will forward packets in a network so their use as a tool to debug a student's configuration (or any kind of network) is undeniable.

Once again, batfish provides us with a function to get this information easily.

In [24]:
result = bf.q.routes(nodes="base_router_27").answer().frame()
display(result)

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
0,base_router_27,default,24.0.10.2/31,interface port_ZURI ip 27.128.21.1,27.128.21.1,port_ZURI,ospf,30,110,None
1,base_router_27,default,24.0.10.2/31,interface BASE-L2 ip 27.160.0.0,27.160.0.0,BASE-L2,ospf,30,110,None
2,base_router_27,default,24.0.10.2/31,interface BASE-L2.10 ip 27.176.0.0,27.176.0.0,BASE-L2.10,ospf,30,110,None
3,base_router_27,default,24.0.10.2/31,interface BASE-L2.20 ip 27.192.0.0,27.192.0.0,BASE-L2.20,ospf,30,110,None
4,base_router_27,default,24.0.12.2/31,interface port_ZURI ip 27.128.21.1,27.128.21.1,port_ZURI,ospf,30,110,None
...,...,...,...,...,...,...,...,...,...,...
631,base_router_27,default,29.203.0.0/24,interface BASE-L2.20 ip 27.192.0.0,27.192.0.0,BASE-L2.20,ospf,34,110,None
632,base_router_27,default,198.0.0.0/24,interface port_ZURI ip 27.128.21.1,27.128.21.1,port_ZURI,ospf,20,110,None
633,base_router_27,default,198.0.0.0/24,interface BASE-L2 ip 27.160.0.0,27.160.0.0,BASE-L2,ospf,20,110,None
634,base_router_27,default,198.0.0.0/24,interface BASE-L2.10 ip 27.176.0.0,27.176.0.0,BASE-L2.10,ospf,20,110,None


Interestingly, it seems that batfish mislabeled some of the entries in the dataframe. You can clearly see that some IPs that are not part of AS 27 are being accessed via the ospf protocol. Furthermore, this routing table seems too big. Router aren't supposed to know every address of every other AS.

When looking at Batfish's documentation, we can see that this phenomenon isn't supposed to happen, the protocols are usually labelled correctly and the tables aren't this large. This problem might be explained by the way our files were parsed. Indeed, in most of the examples provided by the batfish developpers, the files used are cisco configuration files. In our case, we converted frrouting files to a cumulus format so batfish can read them. This bug might be a side effect of this conversion.

Despite all this, the above dataframe can still be used if filtered correctly. We're going to only check the entries of the routing table that lead to the loopback addresses of other routers.

In [25]:
show(result[result["Network"].str.contains("27.144.")])

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
356,base_router_27,default,27.144.1.0/32,interface port_ZURI ip 27.128.21.1,27.128.21.1,port_ZURI,ospf,10,110,None
357,base_router_27,default,27.144.1.0/32,interface BASE-L2 ip 27.160.0.0,27.160.0.0,BASE-L2,ospf,10,110,None
358,base_router_27,default,27.144.1.0/32,interface BASE-L2.10 ip 27.176.0.0,27.176.0.0,BASE-L2.10,ospf,10,110,None
359,base_router_27,default,27.144.1.0/32,interface BASE-L2.20 ip 27.192.0.0,27.192.0.0,BASE-L2.20,ospf,10,110,None
360,base_router_27,default,27.144.2.0/24,interface lo,AUTO/NONE(-1l),lo,connected,0,0,None
361,base_router_27,default,27.144.3.0/32,interface port_GENE ip 27.128.32.0,27.128.32.0,port_GENE,ospf,10,110,None
362,base_router_27,default,27.144.4.0/32,interface port_ZURI ip 27.128.21.1,27.128.21.1,port_ZURI,ospf,20,110,None
363,base_router_27,default,27.144.4.0/32,interface port_GENE ip 27.128.32.0,27.128.32.0,port_GENE,ospf,20,110,None
364,base_router_27,default,27.144.4.0/32,interface BASE-L2 ip 27.160.0.0,27.160.0.0,BASE-L2,ospf,20,110,None
365,base_router_27,default,27.144.4.0/32,interface BASE-L2.10 ip 27.176.0.0,27.176.0.0,BASE-L2.10,ospf,20,110,None


<br><font size=5 color=#000000> <b>PART 4 - INTER AS CONFIGURATION</b> </font> <br>

In this section, we will focus on the second main part of the mini-internet project: the BGP configuration between ASes. We will first cover the basic analysis opportunities offered by batfish, and then we will delve more into the details of this tool in the context to verify the policies the student created to advertise their routes.

### Basic BGP properties

So, as stated in the previous introduction, we need to take a look at all the properties listed by batfish about the BGP configuration in order to figure out what could be used to verify the configurations.

Again, we will have to limit ourselves about the features we want to analyse. Here are what we could get as information from each BGP connexion:

In [26]:
list(bf.q.bgpProcessConfiguration().answer().frame().columns)[2:]

['Router_ID',
 'Confederation_ID',
 'Confederation_Members',
 'Multipath_EBGP',
 'Multipath_IBGP',
 'Multipath_Match_Mode',
 'Neighbors',
 'Route_Reflector',
 'Tie_Breaker']

From this list we will only use a subset composed of `Router_ID`, `Neighbors` and `Tie_Breaker`. The choice to discard the `Confederation` facts as well as the `Route_Reflector` is simply based on the way our network was design, where these features weren't used. On the other hand, we could have much more interesting information by getting the `Neighbors` property. More information about each property can be found, as always, inside the <a href=https://batfish.readthedocs.io/en/latest/notebooks/configProperties.html#BGP-Process-Configuration> documentation of batfish </a>.

Let's take a look at the head of the answered frame to see if we could infer some interesting fact of our network.

In [27]:
bgpProcConfig = bf.q.bgpProcessConfiguration(properties='/Router_ID|Neighbors|Tie_Breaker/').answer().frame()
bgpProcConfig.head(6)

,Node,VRF,Router_ID,Neighbors,Tie_Breaker
0,muni_router_26,default,26.200.5.0,"['23.11.0.7', '26.200.1.0', '26.200.2.0', '26.200.3.0', '26.200.4.0', '26.200.6.0', '26.200.7.0', '26.200.8.0']",ARRIVAL_ORDER
1,muni_router_24,default,24.155.0.1,"['24.151.0.1', '24.152.0.1', '24.153.0.1', '24.154.0.1', '24.156.0.1', '24.157.0.1', '24.158.0.1', '179.0.9.1']",ARRIVAL_ORDER
2,muni_router_27,default,27.144.5.0,"['27.128.50.2', '27.144.1.0', '27.144.2.0', '27.144.3.0', '27.144.4.0', '27.144.6.0', '27.144.7.0', '27.144.8.0']",ARRIVAL_ORDER
3,vien_router_25,default,25.127.0.7,"['25.127.0.1', '25.127.0.2', '25.127.0.3', '25.127.0.4', '25.127.0.5', '25.127.0.6', '25.127.0.8', '180.122.0.122']",ARRIVAL_ORDER
4,muni_router_30,default,30.155.0.1,"['30.151.0.1', '30.152.0.1', '30.153.0.1', '30.154.0.1', '30.156.0.1', '30.157.0.1', '30.158.0.1', '179.0.68.1']",ARRIVAL_ORDER
5,mila_router_28,default,28.158.0.1,"['28.151.0.1', '28.152.0.1', '28.153.0.1', '28.154.0.1', '28.155.0.1', '28.156.0.1', '28.157.0.1', '179.0.71.2']",ARRIVAL_ORDER


While not giving a lot of interesting information at the first look, this first question could be useful to do a kind of pre-check of our entire network.

<div class="alert alert-info">
<b>[Remark] Tie-breaker</b><br>
While not really using the [Tie_Breaker] information during the rest of our report, we wanted to point the default choice made inside our network. Indeed, no configuration file used in this analysis mentioned the use of a specific way to choose between two possible BGP routes. But, here, in batfish, the defaults tie breaker isn't to give more preference to the route from the lowest router id but is to choose the route that was first received. But it is also worth mentionning, again by default, that batfish considers each router connection to do multipath BGP.</div>

First of all, did all the students correctly set up the iBGP connections ? For this, we have to check, for each router, that the number of "Neighbors" starting by the prefix of the AS is effectively 7.

In [28]:
def checkAS(rid, neigh):
    return np.array([addr.split(".")[0] == rid for addr in neigh]) 

allConnection = True
for i in range(len(bgpProcConfig.index)):
    rid = bgpProcConfig.iloc[i]["Router_ID"]
    neigh = np.array(bgpProcConfig.iloc[i]["Neighbors"])
    allConnection = checkAS(rid.split(".")[0], neigh).sum() >= 7
    if not allConnection:
        print(f"This router_id doesn't follow the rule: {rid}")
        break
    
# All the iBGP connections for each router and each student did it for each of his/her router
allConnection and len(bgpProcConfig) == 8*7

True

Doing this operation needed a little more data processing techniques than for the previous results we could obtain directly. But our objective is reached, we can confirm that every student correctly initialized all the iBGP connections between each of her/his routers. 

But here we only check some quantitative values to identify the configuration. We could also want to have a more accurate and qualitative content to analyse. This can be done if we want to further analyse on of the criteria we need to satisfy inside our network. Indeed, the iBGP peers should be connected to each other by using their loopback addresses. The relative question to see if a student comply to this requirement could be to look if the correct addresses were used. To avoid too much computing on multiple dataframes, we could find another question from batfish that could give us the desired result. While checking the documentation, we found this <a href=https://batfish.readthedocs.io/en/latest/notebooks/configProperties.html#BGP-Peer-Configuration> interesting batfish question</a>. Let's try to run it in the next code snippet.

In [29]:
bgpPeer = bf.q.bgpPeerConfiguration(properties='/Local_AS|Local_IP|Remote_AS|Remote_IP|Import_Policy|Export_Policy/').answer().frame()
display(bgpPeer.head(6))

,Node,VRF,Local_AS,Local_IP,Local_Interface,Remote_AS,Remote_IP,Import_Policy,Export_Policy
0,lyon_router_27,default,27,27.144.6.0,None,27,27.144.7.0,[],[]
1,lyon_router_29,default,29,29.156.0.1,None,29,29.158.0.1,[],[]
2,vien_router_29,default,29,29.157.0.1,None,29,29.155.0.1,[],[]
3,luga_router_28,default,28,28.154.0.1,None,28,28.152.0.1,[],[]
4,vien_router_26,default,26,26.200.7.0,None,26,26.200.4.0,[],[]
5,luga_router_26,default,26,26.200.4.0,None,26,26.200.2.0,[],[]


Again for ease of reading, we constrained the number of properties to be listed in our dataframes as well as the number of rows. (more information about the other properties can be found in the previous mentioned documentation).

But the most interesting fact in these first 6 lines of the frame is the complete absence of recognized local interface which was the objective of this analysis in order to determine if the students used the loopback one. We can conclude that batfish wasn't able to process this information for us, and thus we will have to do the computation by ourselves. For this we will reuse the `iface_prop` frame we created in the second section while analysing the properties of all the interfaces in our network.

But before going further, there is a problem with the previous frame. By only displaying the head of this table, we aren't able to figure out the presence of eBGP connection in it. We thus have to first filter the connections to only get the iBGP ones.

In [30]:
ibgpPeer = bgpPeer[bgpPeer["Local_AS"] == bgpPeer["Remote_AS"].astype(int)]

def checkIfLoopback(addr, router_name, frame):
    loopaddr = frame[frame["Interface"].astype(str) == f"{router_name}[lo]"]["Primary_Address"].values[0]
    return str(loopaddr).split("/")[0] == addr

usedLoopback = True
for i in range(len(ibgpPeer.index)):
    rname = ibgpPeer.iloc[i]["Node"]
    addr_local = ibgpPeer.iloc[i]["Local_IP"]
    usedLoopback = checkIfLoopback(addr_local, rname, iface_prop)
    if not usedLoopback:
        print(f"This router name doesn't follow the rule: {rname}")
        break

usedLoopback

True

The result is true, every student worked well on this part and really take care to use the loopback addresses as required. In other words, no student forgets about the `update-source lo` during the configuration of each of their iBGP configuration.

<div class="alert alert-info">
<b>[Remark] Import and Export policy</b><br>
In the previous frame displayed, we could easily see that the policy columns are composed of empty list. Will this mean that no policies were defined by the students ? Obviously no. This is only beacuse we limited our representation of the frame by display the 6 first lines and by chance (or not) they were all 6 iBGP connections. If we do the displaying but only with the eBGP peering, the policies appear (and this will be useful for the second section of this part).</div>

In [31]:
ebgpPeer = bgpPeer[bgpPeer["Local_AS"] != bgpPeer["Remote_AS"].astype(int)]
display(ebgpPeer.head(6))

,Node,VRF,Local_AS,Local_IP,Local_Interface,Remote_AS,Remote_IP,Import_Policy,Export_Policy
6,lyon_router_28,default,28,179.0.70.1,None,30,179.0.70.2,['ACCEPT_ALL'],[]
18,vien_router_29,default,29,180.122.0.29,None,122,180.122.0.122,['COMU'],['MAP-OUT']
29,lyon_router_29,default,29,179.0.72.1,None,31,179.0.72.2,['ACCEPT_ALL_IN'],['ACCEPT_ALL']
30,muni_router_26,default,26,23.11.0.6,None,23,23.11.0.7,['MAP_IN'],['MAP_OUT']
32,luga_router_30,default,30,179.0.74.2,None,29,179.0.74.1,['ACCEPT_ALL'],['MAP_OUT']
38,luga_router_26,default,26,26.250.2.2,None,25,26.250.2.3,['MAP_IN'],['MAP_OUT']


Before moving to the policy analysis, we wanted to conclude this section by showing two more questions (and their results) that could be interesting for some debugging when reviewing the projects of the students. We would not talk much about all the properties or information that could be inferred by using these questions. We want to get the most global view over the possibilities of batfish for analysing the entire mini-internet project. The links to the documentation is always given if you want more details about the information included in each question.

First of all, the session compatibility question `bgpSessionCompatibility` (<a href=https://batfish.readthedocs.io/en/latest/notebooks/routingProtocols.html#BGP-Session-Compatibility>for docs click on this link</a>), this question will mainly show the same information that could be obtained by the `bgpPeerConfiguration` one but will add the possibility to see the status of the connection between 2 peers. This could be `UNIQUE_MATCH` if there is no conflict or problem, `UNKOWN_REMOTE` if either the remote AS or IP address isn't known and `NO_LOCAL_IP` if the AS is recognized but the link seems to be one directional. Here is a little overview of these results for the ebGP connections.

In [32]:
bgpCompat = bf.q.bgpSessionCompatibility().answer().frame()
ebgpCompat = bgpCompat[bgpCompat["Local_AS"] != bgpCompat["Remote_AS"].astype(int)]
ebgpCompat.drop(["Remote_Interface", "Address_Families", "Local_Interface"], axis=1).head(6)

,Node,VRF,Local_AS,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Configured_Status
7,base_router_24,default,24,179.0.11.2,22,None,179.0.11.1,EBGP_SINGLEHOP,UNKNOWN_REMOTE
8,base_router_25,default,25,23.11.0.2,23,None,23.11.0.1,EBGP_SINGLEHOP,UNKNOWN_REMOTE
16,base_router_26,default,26,24.250.0.5,24,lyon_router_24,24.250.0.4,EBGP_SINGLEHOP,UNIQUE_MATCH
24,base_router_27,default,27,25.43.6.3,25,lyon_router_25,25.43.6.2,EBGP_SINGLEHOP,UNIQUE_MATCH
25,base_router_27,default,27,None,25,None,25.46.6.2,EBGP_SINGLEHOP,NO_LOCAL_IP
40,base_router_28,default,28,179.0.65.2,26,lyon_router_26,179.0.65.1,EBGP_SINGLEHOP,UNIQUE_MATCH


Again, the responsible for the AS 27 has an explanation for the `NO_LOCAL_IP` problem. This is link to the same problem of not enough cleaning inside the configuration of the routers, due to a miss click between 3 and 6, so he created two connections for the same AS.

The second interesting debugging question is `bgpSessionStatus` which allows to indicate if the BGP session was successfully established or not (<a href=https://batfish.readthedocs.io/en/latest/notebooks/routingProtocols.html#BGP-Session-Status>more details here</a>). 

In [33]:
bgpStatus = bf.q.bgpSessionStatus().answer().frame()
ebgpStatus = bgpStatus[bgpCompat["Local_AS"] != bgpStatus["Remote_AS"].astype(int)]
ebgpStatus.drop(["Remote_Interface", "Address_Families", "Session_Type", "Local_Interface"], axis=1).head(6)

,Node,VRF,Local_AS,Local_IP,Remote_AS,Remote_Node,Remote_IP,Established_Status
7,base_router_24,default,24,179.0.11.2,22,None,179.0.11.1,NOT_COMPATIBLE
8,base_router_25,default,25,23.11.0.2,23,None,23.11.0.1,NOT_COMPATIBLE
16,base_router_26,default,26,24.250.0.5,24,lyon_router_24,24.250.0.4,ESTABLISHED
24,base_router_27,default,27,25.43.6.3,25,lyon_router_25,25.43.6.2,ESTABLISHED
25,base_router_27,default,27,None,25,None,25.46.6.2,NOT_COMPATIBLE
40,base_router_28,default,28,179.0.65.2,26,lyon_router_26,179.0.65.1,ESTABLISHED


We directly see that every connection that was under a problem identifier inside the previous frame is also indicated to be not established inside this frame. More than being just a rehearsal of the previous question, this result could be interesting to see if a student only set its bgp connection in a direction but not in the other.

<div class="alert alert-info">
<b>[Remark] AS representation of batfish</b><br>
You may have notice the [astype(int)] figuring each time we have to limit the representation to the eBGP or the iBGP connections. This is because batfish has decided to format [Local_AS] and [Remote_AS] differently causing the first to be represented as an integer and the second as a string. This could lead to some difficulties when we want to compute something and you are not aware of it. This problem arose also when we want to compare the loopback addresses with the used address for iBGP. The interface field inside the [iface_prop] is an inner batfish object while the addresses in the BGP frames were represented as a string. A simple conversion generally solves the problem but keep in mind that some comparison seeming straightforward could lead to biased result due to the type difference.</div>

### Analysing the eBGP policies

After all these considerations about the complete properties analysis of the BGP sessions and connections, we could try to create a method to automatically identify if an AS only receives the routes provided by its customers, its peers and its providers but doesn't advertise the routes from these peers and providers. Globally, there are 2 main things to check: the setting up local-preference to prefer the customer routes from the peer routes from the provider ones and the filtering of the route out-bound. This is the topic of this section.

But in order to do further analysis, we would need to get the policies for each router because batfish will need these to process the information. We will first define a function to get them.

In [34]:
def getPolicies(node):
    ebgp = ebgpPeer[ebgpPeer["Node"] == node]
    in_policy = ebgp["Import_Policy"].values[0]
    out_policy = ebgp["Export_Policy"].values[0]
    return {"node": node, "in": in_policy, "out": out_policy}

This function using a string representing the name of a node and the ebgpPeer configuration frame we defined above will be useful for the rest of this analysis. 

However, another problem arisen quickly: how to let batfish know which AS is the client, the peer or the provider of whom. We could try to hardcode this value for each analysis but this would be really long, unreadable and in opposition with our envy to automatize this verification. The only solution in order to not pollute our code, is to write the relations and the graph representation of our network inside an additional file. But writing a gigantic file with the complete adjacency list would be too much error-prone and difficult to maintain.

Hopefully here, the configuration of each AS was equivalent (base name of the routers) and the format of the interfaces leading to other ASes was always of the same type with the same connections' relationship.

For example, the `MILA_router` of an AS Y is always connected to an AS X which would be its customer and this by using the interface `ext_X_NAME`. While in the X AS, this connection will end up in the router `NAME_router` having `ext_Y_MILA` as interface for this connection. Then since each router name is related to a type of economical relationship. `MILA` and `LYON` are connected to external customers, `MUNI` and `BASE` to providers and `LUGA` to peers. This we highly simplify our analysis, and we will use this strategy since we target an overview of the possibility to verify the mini-internet project.

<div class="alert alert-info">
<b>[Remark] Being completely context free</b><br>
We first thought about a way to be completely out of any form of context by creating a sort of additional configuration file to specify for each node the link with the other nodes and the relationship that links each of these nodes. Since it would add a lot of overhead to our current analysis without adding an efficient behaviour, we decided to let this as en improvement for further work on batfish.</div>

Let's define some reference objects to identify the relationships between ASes. Here we choose to display the role of the router inside the AS according to its external link. By saying that MILA is a "provider" we simpy indicate that it is connected to customers. So the local preference for the route learned by these clients on this "provider" router, should have the highest local-pref. We take this approach because we are sure of the current routers connected to external links, and we are sure about the role they have. But the arriving router in the external AS could be different (like for example the AS 30 have an external connection with ZURI router inside the AS 32 which was a stub AS with a different configuration form)

<div class="alert alert-warning">
<b>[Warning] Name convention</b><br>
Don't be confused during the following paragraphs of our analysis. We use the role of the routers inside the current AS and not the type of external AS to consider the correct value of local-pref. So when we say that provider sets a high local-pref, we mean "as a provider, I correctly set a high local-pref to the routes advertised by my customers". Accordingly, syaing "customer sets a low local-pref" we mean "as a customer, I effectively set a low local-pref to the routes advertised by my providers"</div>

In [35]:
relation = {
    "MILA": "provider",
    "LYON": "provider",
    "LUGA": "peer",
    "MUNI": "customer",
    "BASE": "customer",
}

ases = ["24", "25", "26", "27", "28", "29", "30"]

We can now define a function that will do the computation for us. The mechanism is simple, we will go through each node having external BGP connections, we will then get the interface responsible for this connection. Parsing this interface name will allow us to get the AS number of the neighbor. With this, we will be able to generate a virtual BGP route from this remote AS through our current AS. Then we will fetch all the information about the changes done to the BGP state from this source. We store the change on a local pref level. Finally, we compare these values to be sure that, if the current AS is a provider (the remote a customer so), the local pref is always the highest while if the AS is a customer (remote a provider), we always give the lowest local-pref to this route (peer route should be in the middle).

In [36]:
def getASremote(AS, name):
    ebgp = ebgpPeer[ebgpPeer["Node"] == f"{name}_router_{AS}"]
    remote_as = ebgp["Remote_AS"].values[0]
    return remote_as

def verifyLocalPref(AS):
    local_pref = []
    as_id = AS
    res = {}
    for name in relation:
        ext_iface = bf.q.interfaceProperties(nodes=f"/{name.lower()}_router_{as_id}/",
                                             interfaces="/ext/",
                                             properties="Active").answer().frame()
        ext_iface = ext_iface["Interface"].values[0]
        ext_if_name = ext_iface.interface.split("_")
        ext_as = ext_if_name[1]

        remote_prefix = f"{ext_as}.0.0.0/8"
        remote_origin = f"{ext_as}.0.0.0"
        inroute = BgpRoute(network=remote_prefix,
                           originatorIp=remote_origin,
                           originType="egp",
                           protocol="bgp")

        test = bf.q.testRoutePolicies(nodes=f"{name.lower()}_router_{as_id}",
                                      policies=getPolicies(f"{name.lower()}_router_{as_id}")["in"][0], 
                                      direction="in", 
                                      inputRoutes=[inroute]).answer().frame()

        diffs = test["Difference"].values[0].diffs
        locpref = False
        for d in diffs:
            if d.fieldName == 'localPreference':
                locpref = True
                if relation[name] in res :
                    res[relation[name]] = np.append(res[relation[name]], int(d.newValue))
                else :
                    res[relation[name]] = np.array([int(d.newValue)])

        if diffs == [] or not locpref:
            res[relation[name]] = np.array([0])

    # Provider receives its routes from a client so set a high local-pref
    # Peer receives its routes from a peer so set a middle local-pref
    # Client receives its routes from a provider so set a low local-pref
    prov_check = min(res["provider"]) > max(res["peer"])
    peer_check = min(res["peer"]) > max(res["customer"])

    return prov_check and peer_check    

result = pd.DataFrame(columns=ases, index=["Correct Local Pref"])
for AS in ases :
    result[AS] = verifyLocalPref(AS)

display(result)

,24,25,26,27,28,29,30
Correct Local Pref,True,True,True,True,True,True,True


After applying our function on all the ASes in our network, we effectively see that every student correctly set up the local-preference field. This is the first "automated" test enabling a complete autonomous verification of the configuration given by the student and this only by using batfish and its virtualized routes.

<div class="alert alert-warning">
<b>[Warning] Specificity of our analysis</b><br>
Again we want to insist on the fact we choose to focus on the mini-internet project so the previous function is completely bound to the actual configuration of our network. Here we take advantage of the structures and information contained inside the external interface name. Furthermore, we used the fact that for example [MUNI] is alsways connected to a provider which would be [MILA]. Of course, we should have adpoted another method is these cases weren't present. We would have used some additional files de describe which route is connected to which AS with which relationship. We should also have used a complete search for remote AS number inside the ebgpPeer frame. For simplicity according to our network, we will stay and bask in our current approach.
</div>

Let's now see the out-bound addresses permitted and denied. We will first cover the fact that each AS have to advertise its own prefix. For this, we will see if no AS is denying its base address.

In [37]:
def verifyOwnAddress(AS) :
    ownPrefix = f"{AS}.0.0.0/8"
    ownOrigin = f"{AS}.0.0.0"
    outRoute = BgpRoute(network=ownPrefix,
                        originatorIp=ownOrigin,
                        originType="egp",
                        protocol="bgp")
    
    advertised = True
    for name in relation:
        policies = getPolicies(f"{name.lower()}_router_{AS}")
        out_policy = policies["out"][0] if len(policies['out']) > 0 else ""

        res = bf.q.testRoutePolicies(nodes=f"{name.lower()}_router_{AS}",
                                         policies=out_policy,
                                         direction="out",
                                         inputRoutes=[outRoute]).answer().frame()
    
        res = res[res["Action"].map(lambda a: a == "PERMIT")]
        
        advertised = advertised and (len(res.index) >= 1)
    return advertised

result = pd.DataFrame(columns=ases, index=["Advertise own prefix"])
for AS in ases :
    result[AS] = verifyOwnAddress(AS)

display(result)

,24,25,26,27,28,29,30
Advertise own prefix,False,True,True,True,True,True,True


Nearly all the ASes have an affirmative answer to the previous self-coded question but not the AS 24. Would it be possible that the student responsible for this part of the mini-internet project forgot to advertise her/his prefix? Before concluding a misconfiguration of the policies, let's check the outgoing policies for this AS in one of its router like for example `mila_router`.

In [38]:
route_24_mila = BgpRoute(network="24.0.0.0/8", originatorIp="24.0.0.0", originType="egp", protocol="bgp")
bf.q.testRoutePolicies(nodes="mila_router_24", direction="out", 
                       inputRoutes=[route_24_mila]).answer().frame().head(3)

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference,Trace
0,mila_router_24,PREFIX_COMMUNITY,"BgpRoute(network='24.0.0.0/8', originatorIp='24.0.0.0', originType='egp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, nextHopIp=None, sourceProtocol=None, tag=0, weight=0)",PERMIT,"BgpRoute(network='24.0.0.0/8', originatorIp='24.0.0.0', originType='egp', protocol='bgp', asPath=[], communities=['24:8'], localPreference=0, metric=0, nextHopIp=None, sourceProtocol=None, tag=0, weight=0)","BgpRouteDiffs(diffs=[BgpRouteDiff(fieldName='communities', oldValue='[]', newValue='[24:8]')])",- Matched route-map PREFIX_COMMUNITY entry 10
1,mila_router_24,PROVIDER_IN,"BgpRoute(network='24.0.0.0/8', originatorIp='24.0.0.0', originType='egp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, nextHopIp=None, sourceProtocol=None, tag=0, weight=0)",PERMIT,"BgpRoute(network='24.0.0.0/8', originatorIp='24.0.0.0', originType='egp', protocol='bgp', asPath=[], communities=['24:25'], localPreference=150, metric=0, nextHopIp=None, sourceProtocol=None, tag=0, weight=0)","BgpRouteDiffs(diffs=[BgpRouteDiff(fieldName='communities', oldValue='[]', newValue='[24:25]'), BgpRouteDiff(fieldName='localPreference', oldValue='0', newValue='150')])",- Matched route-map PROVIDER_IN entry 10
2,mila_router_24,PROVIDER_OUT,"BgpRoute(network='24.0.0.0/8', originatorIp='24.0.0.0', originType='egp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, nextHopIp=None, sourceProtocol=None, tag=0, weight=0)",DENY,None,None,


Effectively, the `mila_router` is denying every route with this prefix when going through the `PROVIDER_OUT` route-map. But this doesn't mean a complete denying of the route. By seeing the other route-maps present in this router configuration, we see the `PREFIX_COMMUNITY` route-map transforming the community values of the received route. By a little analysis of the router configuration file, we see that `PROVIDER_OUT`is normally permitting all the routes with this value of community. So, the next code snippet test to advertise the same route but with the correct community values.

In [39]:
commu_24_mila = BgpRoute(network="24.0.0.0/8", originatorIp="24.0.0.0", originType="egp", protocol="bgp",
                         communities=["24:8"])
bf.q.testRoutePolicies(nodes="mila_router_24", direction="out", 
                       inputRoutes=[commu_24_mila]).answer().frame().head(3)

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference,Trace
0,mila_router_24,PREFIX_COMMUNITY,"BgpRoute(network='24.0.0.0/8', originatorIp='24.0.0.0', originType='egp', protocol='bgp', asPath=[], communities=['24:8'], localPreference=0, metric=0, nextHopIp=None, sourceProtocol=None, tag=0, weight=0)",PERMIT,"BgpRoute(network='24.0.0.0/8', originatorIp='24.0.0.0', originType='egp', protocol='bgp', asPath=[], communities=['24:8'], localPreference=0, metric=0, nextHopIp=None, sourceProtocol=None, tag=0, weight=0)",BgpRouteDiffs(diffs=[]),- Matched route-map PREFIX_COMMUNITY entry 10
1,mila_router_24,PROVIDER_IN,"BgpRoute(network='24.0.0.0/8', originatorIp='24.0.0.0', originType='egp', protocol='bgp', asPath=[], communities=['24:8'], localPreference=0, metric=0, nextHopIp=None, sourceProtocol=None, tag=0, weight=0)",PERMIT,"BgpRoute(network='24.0.0.0/8', originatorIp='24.0.0.0', originType='egp', protocol='bgp', asPath=[], communities=['24:25'], localPreference=150, metric=0, nextHopIp=None, sourceProtocol=None, tag=0, weight=0)","BgpRouteDiffs(diffs=[BgpRouteDiff(fieldName='communities', oldValue='[24:8]', newValue='[24:25]'), BgpRouteDiff(fieldName='localPreference', oldValue='0', newValue='150')])",- Matched route-map PROVIDER_IN entry 10
2,mila_router_24,PROVIDER_OUT,"BgpRoute(network='24.0.0.0/8', originatorIp='24.0.0.0', originType='egp', protocol='bgp', asPath=[], communities=['24:8'], localPreference=0, metric=0, nextHopIp=None, sourceProtocol=None, tag=0, weight=0)",PERMIT,"BgpRoute(network='24.0.0.0/8', originatorIp='24.0.0.0', originType='egp', protocol='bgp', asPath=[], communities=['24:8'], localPreference=0, metric=0, nextHopIp=None, sourceProtocol=None, tag=0, weight=0)",BgpRouteDiffs(diffs=[]),- Matched route-map PROVIDER_OUT entry 10


Now we can clearly see this route being permitted without any concern. This another flaw of batfish. When we want to compute and verify policies rules, we are forced to use the correct route with all the parameters fixed by ourselves. Testing route policies doesn't do a kind of traceroute to apply all the modifications on the advertisements, we have to be aware of the format our route will have in order to correctly verify its communication to the other ASes. This phenomenon is also illustrated in the next example about the client addresses.

We first wanted to use the AS path to identify the route denied from the permitted ones. But, after a tedious work on this possibility and a search on the issues page of batfish, we saw that this type of analysis would be completely unfeasible in the case of our network. Indeed, we would have used the `searchRoutePolicies` question for this purpose but this function is relatively new and the batfish documentation says itself that it could possibly fail. Furthermore, some issues are opened on the problem of eBgp not being advertised over iBgp leading to problem for getting all the policies. Finally, since the routes have to be defined exactly to be processed, including the exact AS path (the regex argument for using AS path fails nearly everytime), we cannot easily, in an autonomous fashion, see if the "economical" relationship of advertisement are strictly respected.

However, we tried to define our own method to verify some part of this last requirement. In the computation below, we will try to figure out if each incoming route from a client is correctly announced on all the other eBGP connections after the application of the transformation required on this route.

In [40]:
providers_router = ["MILA", "LYON"]

def verifyClientAddress(AS):
    advertised = True
    for name in providers_router:
        ext_iface = bf.q.interfaceProperties(nodes=f"/{name.lower()}_router_{AS}/",
                                             interfaces="/ext/",
                                             properties="Active").answer().frame()
        ext_iface = ext_iface["Interface"].values[0]
        ext_if_name = ext_iface.interface.split("_")
        ext_as = ext_if_name[1]
        
        policies = getPolicies(f"{name.lower()}_router_{AS}")
        in_policy = policies["in"][0] if len(policies['in']) > 0 else ""
        
        bgproute = BgpRoute(network="0.0.0.0/0", originatorIp=f"{ext_as}.0.0.0", 
                            originType="egp", protocol="bgp")
        
        res = bf.q.testRoutePolicies(nodes=f"/{name.lower()}_router_{AS}/", policies=in_policy, 
                                     inputRoutes=[bgproute], direction="in").answer().frame()
        
        inroute = res["Output_Route"].values[0]
        
        for other in relation :
            if other == name:
                continue
            
            ext_iface = bf.q.interfaceProperties(nodes=f"/{other.lower()}_router_{AS}/",
                                                 interfaces="/ext/",
                                                 properties="Active").answer().frame()
            ext_iface = ext_iface["Interface"].values[0]
            ext_if_name = ext_iface.interface.split("_")
            ext_as = ext_if_name[1]
            
            policies = getPolicies(f"{other.lower()}_router_{AS}")
            out_policy = policies["out"][0] if len(policies["out"]) > 0 else ""
            
            res = bf.q.testRoutePolicies(nodes=f"/{other.lower()}_router_{AS}/",
                                         policies=in_policy,
                                         inputRoutes=[inroute], direction="out").answer().frame()
        
            advertised = advertised and len(res[res["Action"] == "PERMIT"]) > 0
            if not advertised:
                break
        if not advertised:
                break
    return advertised

result = pd.DataFrame(columns=ases, index=["Advertise client prefix"])
for AS in ases :
    result[AS] = verifyClientAddress(AS)

display(result)

,24,25,26,27,28,29,30
Advertise client prefix,False,False,True,True,True,False,False


This result seems really too bad to be a real and correct one, knowing that each of these ASes were reachable during the mini-internet project. This means that our method isn't completely correct, and we are doing some mistakes somewhere. Indeed, we didn't consider the inter AS transformation of our route that could happen when the ibgp sessions are processing the information. Our result is completely biased and nothing is really worth keeping for a further analysis or even a grading of the students. The only interesting solution left to see if the students used the correct policies is to do some manual traceroutes (by using batfish for example) to see if the route used are the correct ones. Or we could verify the advertised route over eBGP but this functionality is too unstable for now on batfish so this won't be as useful as we may think.

Maybe in the future, using batfish would be an efficient and useful solution to further analyse the bgp policies. So, don't hesitate to often take a look at the new versions of this tool to improve the different aspects of the results you could get by using it.

<br><font size=5 color=#000000> <b>PART 5 - CONCLUSION AND FURTHER USES</b> </font> <br>

In conclusion, Batfish offers interesting tools that could definitely help students to debug their configurations, or teachers to correct them. It's also interesting to note that setting up Batfish is relatively easy and quick. However, this software is not perfect and some of its functionality could still be improved. It should be used with caution in the context of corrections.